In [1]:
import tensorflow as tf
import numpy as np


In [2]:
import io,os,sys,types
from IPython import get_ipython #for import notebook
from nbformat import read #for import notebook
from IPython.core.interactiveshell import InteractiveShell #for import notebook

In [3]:
def find_notebook(fullname, path=None): #for import notebook
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [4]:
class NotebookLoader(object): #for import notebook
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [15]:
sys.meta_path.append(NotebookFinder())
import myprepdata

importing Jupyter notebook from myprepdata.ipynb


ModuleNotFoundError: No module named 'gensim'

In [6]:
'''
class attention_decoder(object):
    def __init__(self, layers, directions  , maxv, hiddensizes , input_v , output_v, en_state, teacher_forcing):
        
        self.grustate = en_state
        self.output = tf.Variable(tf.zeros([self.hiddensize]),dtype='float32')
        self.W1 = tf.Variable(tf.zeros([self.hiddensizes*2, maxv]), dtype='float32')
        self.W2 = tf.Variable(tf.zeros([self.hiddensizes*2, self.hiddensizes]), dtype='float32')
        self.b1 = tf.Variable(tf.zeros([maxv]), dtype='float32')
        self.b2 = tf.Variable(tf.zeros([self.hiddensizes]), dtype='float32')
        i = 0
        with tf.variable_scope('input') as scope:
            self.input = tf.placeholder(dtype='float32', shape=[1, self.maxl, self.hiddensize])
            

        with tf.variable_scope('gru') as scope:
            self.gru = tf.nn.rnn_cell.GRUCell(num_units = self.hiddensize)
            if teacher_forcing:
                while(i < maxv): #teacher forcing
                    self.grustate, self.output =self.gru.call(self.grustate, tf.slice(output_v,[0,0,0],[0,i,self.hiddensize]))
                    i = i + 1
                i = 0
            else:
                while(i < maxv): #not teacher forcing
                    self.grustate, self.output =self.gru.call(self.grustate, self.output )
                    
                    word = myprepdata.findwordsfromvec(self.output)
                    self.output = myprepdata.findvecfromwords(word)
                    attn = tf.concat(0,[self.output, self.grustate])#concat and linear
                    attn = tf.add(tf.matmul(attn , self.W),b1) # needs W initialize
                    attn = tf.nn.softmax(tf.nn.relu(attn))
                    bmm = tf.matmul(attn, input_v)
                    
                    attn_com = tf.concat(0,[bmm, self.output] )
                    attn_com = tf.add(tf.matmul(attn_com,w2),b2)
                    self.output = tf.relu(attn_com)
                    
                    i = i +1
                i = 0

                
        with tf.variable_scope('decoder') as scope:
            init_hid = tf.Variable(tf.zeros([100], tf.float32),dtype="float32")
            decell = tf.nn.rnn_cell.GRUCell(num_units=100)
            deoutputs, _destates = tf.nn.dynamic_rnn(decell, inputs = inputv, dtype=tf.float32)
   
    def find_mostsim(self,inputv):
        return
        
    def gru_decoder(self):
        return
    
    def hidden_init(self):
        return sess.run(tf.global_variables_initializer())
'''

'\nclass attention_decoder(object):\n    def __init__(self, layers, directions  , maxv, hiddensizes , input_v , output_v, en_state, teacher_forcing):\n        \n        self.grustate = en_state\n        self.output = tf.Variable(tf.zeros([self.hiddensize]),dtype=\'float32\')\n        self.W1 = tf.Variable(tf.zeros([self.hiddensizes*2, maxv]), dtype=\'float32\')\n        self.W2 = tf.Variable(tf.zeros([self.hiddensizes*2, self.hiddensizes]), dtype=\'float32\')\n        self.b1 = tf.Variable(tf.zeros([maxv]), dtype=\'float32\')\n        self.b2 = tf.Variable(tf.zeros([self.hiddensizes]), dtype=\'float32\')\n        i = 0\n        with tf.variable_scope(\'input\') as scope:\n            self.input = tf.placeholder(dtype=\'float32\', shape=[1, self.maxl, self.hiddensize])\n            \n\n        with tf.variable_scope(\'gru\') as scope:\n            self.gru = tf.nn.rnn_cell.GRUCell(num_units = self.hiddensize)\n            if teacher_forcing:\n                while(i < maxv): #teacher fo

In [7]:
gru = tf.nn.rnn_cell.GRUCell

In [8]:
#c = tf.add(c, b)
with tf.variable_scope('gru') as scope:
    a = tf.Variable([[1,0,0,0]],dtype="float32")
    b = tf.Variable([[0,1,0,0]],dtype="float32")
    k = tf.assign(b,tf.subtract(b,a))
    c = tf.Variable([[0,0,1,0]],dtype="float32")
print(a[0][2])
i = 0

with tf.variable_scope('gruu') as scope:
    while(i < 10):
        c = tf.add(b,c)
        c = tf.add(b,c)
        i = i + 1
    #b = tf.add(a,b)
    
with tf.variable_scope('gruuu') as scope:
    ppp = tf.assign(a,tf.subtract(b,a))
    qqq = tf.assign(b,tf.subtract(b,c))
    a_v = tf.summary.histogram("a",a)#not drowed in graph
    b_v = tf.summary.histogram("b",b)
    c_v = tf.summary.histogram("c",c)


Tensor("strided_slice_1:0", shape=(), dtype=float32)


In [9]:
'''
a0 = tf.constant([[[1, 1, 1], [2, 2, 2]], [[3, 3, 3], [4, 4, 4]], [[5, 5, 5], [6, 6, 6]]], dtype = 'float32')


b0 = tf.slice(a , [0, 0, 0], [3, 2, 3])
b1 = tf.slice(a , [0, 0, 1], [1, 1, 2])
b2 = tf.slice(a , [0, 0, 0], [1, 1, 3])
b3 = tf.slice(a , [0, 0, 1], [-1, -1, 2])
b4 = tf.slice(a , [0, 0, 0], [1, 0, 2])
b5 = tf.slice(a , [0, 0, 0], [1, 0, 3])
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run([b,b1,b2,b3,b4,b5])
'''

"\na0 = tf.constant([[[1, 1, 1], [2, 2, 2]], [[3, 3, 3], [4, 4, 4]], [[5, 5, 5], [6, 6, 6]]], dtype = 'float32')\n\n\nb0 = tf.slice(a , [0, 0, 0], [3, 2, 3])\nb1 = tf.slice(a , [0, 0, 1], [1, 1, 2])\nb2 = tf.slice(a , [0, 0, 0], [1, 1, 3])\nb3 = tf.slice(a , [0, 0, 1], [-1, -1, 2])\nb4 = tf.slice(a , [0, 0, 0], [1, 0, 2])\nb5 = tf.slice(a , [0, 0, 0], [1, 0, 3])\nsess = tf.Session()\nsess.run(tf.global_variables_initializer())\nsess.run([b,b1,b2,b3,b4,b5])\n"

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./testgru")
    writer.add_graph(sess.graph)
    i = 0
    step = 0
    
    print(sess.run([b]))
    print(sess.run([c]))
    print(sess.run([ppp]))
    print(sess.run([b]))
    print(sess.run([ppp]))
    print(sess.run([a]))
    print(sess.run([b]))
    print(sess.run([ppp]))
    print(sess.run([ppp]))

    
    

    s = sess.run(summary)
    writer.add_summary(s,i)
    i = i + 1

[array([[ 0.,  1.,  0.,  0.]], dtype=float32)]
[array([[  0.,  20.,   1.,   0.]], dtype=float32)]
[array([[-1.,  1.,  0.,  0.]], dtype=float32)]
[array([[ 0.,  1.,  0.,  0.]], dtype=float32)]
[array([[ 1.,  0.,  0.,  0.]], dtype=float32)]
[array([[ 1.,  0.,  0.,  0.]], dtype=float32)]
[array([[ 0.,  1.,  0.,  0.]], dtype=float32)]
[array([[-1.,  1.,  0.,  0.]], dtype=float32)]
[array([[ 1.,  0.,  0.,  0.]], dtype=float32)]


In [14]:
hiddensize = 100
maxv = 484
teacher_forcing = False
with tf.variable_scope('grustate') as scope:
    grustate = tf.Variable(tf.zeros([1,hiddensize]),dtype='float32')
    
with tf.variable_scope('output') as scope:
    output = tf.Variable(tf.zeros([1,hiddensize]),dtype='float32')
    
with tf.variable_scope('var') as scope:
    W1 = tf.Variable(tf.zeros([hiddensize*2, maxv]), dtype='float32')
    W2 = tf.Variable(tf.zeros([hiddensize*2, hiddensize]), dtype='float32')
    b1 = tf.Variable(tf.zeros([maxv]), dtype='float32')
    b2 = tf.Variable(tf.zeros([hiddensize]), dtype='float32')
    i = 0
    
with tf.variable_scope('input') as scope:
    input_v = tf.placeholder(dtype='float32', shape=[ maxv, hiddensize])

with tf.variable_scope('gru') as scope:
    gru = tf.nn.rnn_cell.GRUCell(num_units = hiddensize)
    if teacher_forcing:
        while(i < maxv): #teacher forcing
            grustate, output =self.gru.call(grustate, tf.slice(output_v,[0,0,0],[0,i,hiddensize]))
            i = i + 1
        i = 0
    else:
        while(i < maxv): #not teacher forcing
            grustate, output =gru.call(grustate, output )
            
            word = myprepdata.findwordsfromvec(output.eval())
            np_output = myprepdata.findvecfromwords(word)
            output = tf.Variable(np_output, dtype="float32")
            attn = tf.concat(0,[output, grustate])#concat and linear
            attn = tf.add(tf.matmul(attn , W),b1) # needs W initialize
            attn = tf.nn.softmax(tf.nn.relu(attn))
            bmm = tf.matmul(input_v, attn)

            attn_com = tf.concat(0,[bmm, output] )
            attn_com = tf.add(tf.matmul(attn_com,w2),b2)
            output = tf.relu(attn_com)

            i = i +1
        i = 0


with tf.variable_scope('decoder') as scope:
    init_hid = tf.Variable(tf.zeros([100], tf.float32),dtype="float32")
    decell = tf.nn.rnn_cell.GRUCell(num_units=100)
    deoutputs, _destates = tf.nn.dynamic_rnn(decell, inputs = inputv, dtype=tf.float32)
    
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./testgru")
    writer.add_graph(sess.graph)
    i = 0
    step = 0
    
    print(sess.run([output]))
    
    

    s = sess.run(summary)
    writer.add_summary(s,i)
    i = i + 1

NameError: name 'myprepdata' is not defined